In [ ]:
# Show all outcomes in a cell
from IPython import get_ipython
get_ipython().ast_node_interactivity = 'all'

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ===================================================================
# BƯỚC 0: IMPORT CÁC THƯ VIỆN CẦN THIẾT
# ===================================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Sử dụng mô hình Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

print("Tất cả thư viện đã được import.")

Tất cả thư viện đã được import.


In [ ]:
# ===================================================================
# BƯỚC 1: TẢI DỮ LIỆU
# ===================================================================
try:
    df = pd.read_csv('/content/drive/MyDrive/train.csv')
    df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
    print("\nTải dữ liệu train.csv và test.csv thành công.")
except FileNotFoundError:
    print("\nLỖI: Không tìm thấy file train.csv hoặc test.csv. Vui lòng tải file lên Colab.")
    exit()

test_ids = df_test['ID']


Tải dữ liệu train.csv và test.csv thành công.


In [ ]:
# ===================================================================
# BƯỚC 2: HÀM TIỀN XỬ LÝ LOGIC RIÊNG VÀ FEATURE ENGINEERING CƠ BẢN
# ===================================================================
def initial_data_prep(dataframe):
    df_copy = dataframe.copy()
    if 'HasParking' in df_copy.columns:
        df_copy.loc[df_copy['HasParking'] == 0, 'ParkingType'] = 'None'
        df_copy['ParkingType'].fillna('Mechanical', inplace=True)
    if 'YearRenovated' in df_copy.columns and 'YearBuilt' in df_copy.columns:
        df_copy.loc[df_copy['YearRenovated'] == 0, 'YearRenovated'] = df_copy['YearBuilt']
        invalid_condition = df_copy['YearRenovated'] < df_copy['YearBuilt']
        df_copy.loc[invalid_condition, 'YearRenovated'] = df_copy.loc[invalid_condition, 'YearBuilt']
    current_year = 2025
    if 'YearBuilt' in df_copy.columns:
        df_copy['BuildingAge'] = current_year - df_copy['YearBuilt']
    if 'YearRenovated' in df_copy.columns:
        df_copy['YearsSinceRenovation'] = current_year - df_copy['YearRenovated']
    if 'TotalFloorSquare_sqm' in df_copy.columns:
        df_copy['TotalFloorSquare_sqm_squared'] = df_copy['TotalFloorSquare_sqm'] ** 2
    return df_copy

df = initial_data_prep(df)
df_test = initial_data_prep(df_test)
print("\nĐã thực hiện tiền xử lý và feature engineering cơ bản.")


Đã thực hiện tiền xử lý và feature engineering cơ bản.


/tmp/ipython-input-9-2076331760.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['ParkingType'].fillna('Mechanical', inplace=True)
/tmp/ipython-input-9-2076331760.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

In [ ]:
# ===================================================================
# BƯỚC 3: TÁCH DỮ LIỆU VÀ XÓA CỘT
# ===================================================================
X = df.drop(columns=['Price_JPY', 'listing year', 'listing month'], errors='ignore')
y = df['Price_JPY']
df_test = df_test.drop(columns=['listing year', 'listing month'], errors='ignore')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nĐã chia dữ liệu: {X_train.shape[0]} mẫu train, {X_val.shape[0]} mẫu validation.")


Đã chia dữ liệu: 1900 mẫu train, 476 mẫu validation.


In [ ]:
# ===================================================================
# BƯỚC 4: FEATURE ENGINEERING NÂNG CAO (THEO VỊ TRÍ)
# ===================================================================
print("\nBắt đầu Feature Engineering nâng cao theo vị trí...")
train_temp_for_agg = X_train.copy()
train_temp_for_agg['Price_JPY'] = y_train
train_temp_for_agg['PricePerSqm'] = train_temp_for_agg['Price_JPY'] / train_temp_for_agg['TotalFloorArea_sqm']
ward_aggregates = train_temp_for_agg.groupby('Ward').agg(
    Ward_Avg_PricePerSqm=('PricePerSqm', 'mean'),
    Ward_Median_BuildingAge=('BuildingAge', 'median')
).reset_index()
X_train = pd.merge(X_train, ward_aggregates, on='Ward', how='left')
X_val = pd.merge(X_val, ward_aggregates, on='Ward', how='left')
df_test = pd.merge(df_test, ward_aggregates, on='Ward', how='left')
X_val.fillna(X_train.mean(numeric_only=True), inplace=True)
df_test.fillna(X_train.mean(numeric_only=True), inplace=True)
print("Hoàn tất Feature Engineering nâng cao.")


Bắt đầu Feature Engineering nâng cao theo vị trí...
Hoàn tất Feature Engineering nâng cao.


In [ ]:
# ===================================================================
# BƯỚC 5: XÂY DỰNG PIPELINE TIỀN XỬ LÝ TỰ ĐỘNG
# ===================================================================
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
for col in ['ID', 'YearBuilt', 'YearRenovated']:
    if col in numeric_features:
        numeric_features.remove(col)
ordinal_features = ['InsulationLevel', 'KitchenQuality', 'EnergyEfficiencyRating', 'BuildingDensityZone', 'CustomFeature_4']
ordinal_orders = [['Low', 'Medium', 'High'], ['Standard', 'High-End'], ['E', 'D', 'C', 'B', 'A'], ['Low', 'Medium', 'High'], ['E', 'D', 'C', 'B', 'A']]
nominal_features = X_train.select_dtypes(include=['object']).columns.tolist()
for col in ordinal_features:
    if col in nominal_features: nominal_features.remove(col)
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', RobustScaler())])
ordinal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OrdinalEncoder(categories=ordinal_orders, handle_unknown='use_encoded_value', unknown_value=-1))])
nominal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('nom', nominal_transformer, nominal_features)],
    remainder='drop')
print("\nĐã xây dựng pipeline tiền xử lý.")


Đã xây dựng pipeline tiền xử lý.


In [ ]:
# ===================================================================
# BƯỚC 6: HUẤN LUYỆN VÀ ĐÁNH GIÁ MÔ HÌNH LINEAR REGRESSION
# ===================================================================
# THAY ĐỔI MÔ HÌNH Ở ĐÂY
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression()) # <-- Sử dụng Linear Regression
])

print(f"\n--- Bắt đầu huấn luyện: Linear Regression ---")
pipeline.fit(X_train, y_train)
val_predictions = pipeline.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
print(f"-> Validation RMSE của Linear Regression: {rmse:,.2f} JPY")


--- Bắt đầu huấn luyện: Linear Regression ---


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['CulturalHeritageZone',
                                                   'FloodRiskLevel',
                                                   'DistanceToStation_m',
                                                   'DistanceToSchool_m',
                                                   'DistanceToHospital_m',
                                                   'NoiseLevel',
                                                   'TravelTimeToMajorHub_min',
                                                   'NumberOfTrainLines_at_NearestStation',
                                                   'Res...
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Ward', 'ZoneType',
                                                   'NearbyConstruction',
                                                   'RoadType', 'ViewQuality',
                                                   'PropertyType',
                                                   'ConstructionMaterial',
                                                   'Orientation', 'ParkingType',
                                                   'KitchenType',
                                                   'LightingSystem',
                                                   'OwnershipType', 'SaleType',
                                                   'SaleCondition',
                                                   'Availability', 'ListedBy',
                                                   'CustomFeature_1'])])),
                ('regressor', LinearRegression())])

-> Validation RMSE của Linear Regression: 3,228,617.21 JPY


In [ ]:
# ===================================================================
# BƯỚC 7: DỰ ĐOÁN TRÊN TẬP TEST VÀ TẠO FILE SUBMISSION
# ===================================================================
print("\n--- Tạo file submission bằng Linear Regression ---")
# Huấn luyện lại trên toàn bộ dữ liệu train để có kết quả tốt nhất
print("Huấn luyện lại mô hình trên toàn bộ tập train...")
pipeline.fit(X, y)

# Dự đoán trên tập test
print("Dự đoán trên tập test...")
test_predictions = pipeline.predict(df_test)

# Tạo file submission
submission_df = pd.DataFrame({'Id': test_ids, 'Price_JPY': test_predictions})
submission_df.to_csv('submission.csv', index=False)

print("\n--- HOÀN TẤT ---")
print("Đã tạo file submission.csv thành công! ✅")
print("5 dòng đầu của file submission:")
print(submission_df.head())